In [1]:
import inspect
import random
from collections import defaultdict

PRIMITIVE_TYPES = [str, int, float]

class Symbol(object):
    def __init__(self, values):
        self.parent = None
        self.InitVals(values)
        self.CalcAttributes()
        
    def InitVals(self, values):
        if inspect.isclass(values):
            self.values = []
            for subclass in values.__subclasses__():
                self.values.append(subclass())
        else:
            if type(values) is Symbol:
                self.values = values.values
            elif type(values) is list:
                self.values = values
            else:
                self.values = [values]
        if len(self.values) == 1:
            self._value = self.values[0]
        else:
            self._value = None
        
    def __repr__(self):
        return str(self.val)
    
    def __getattr__(self, attr):
        if attr == 'val':
            if self._value:
                return self._value
            else:
                if len(self.values) == 1:
                    return self.values[0]
                else:
                    return self.values
        raise AttributeError("{} object has no attribute {}".format(self.__class__, attr))
    
    def CalcAttributes(self):
        # TODO: drop deprecated attributes
        attribute_values = defaultdict(list)
        for value in self.values:
            if type(value) in PRIMITIVE_TYPES:
                continue
            for key, val in AttributesDict(value).items():
                if isinstance(val, Symbol):
                    attribute_values[key].extend(val.values)
                elif type(val) is list:
                    attribute_values[key].extend(val)
                else:
                    attribute_values[key].append(val)
        for key, vals in attribute_values.items():
                # implies recursion!
                symbol = Symbol(vals)
                symbol.parent = self
                symbol.attribute_name = key
                setattr(self, key, symbol)
    
    def Observe(self):
        if self._value:
            return self._value
        else:
            return self.Collapse(None)
    
    def Collapse(self, symbol):
        if symbol is None:
            return self.Collapse(Symbol([random.choice(self.values)]))
        # TODO: reject if intersection is 0? some basic rejection 
        # TODO: symbol is type
        
        if type(symbol) not in [Symbol, type]:
            symbol = Symbol(symbol)

        # adopt new values
        self.InitVals(Subset(self, symbol))

        # recalculate my attributes
        self.CalcAttributes()

        # restrict my values to only those whose attributes are also consistent with symbol
        if self.parent is not None:
            # self.parent.Collapse(self.parent)
            self.parent.Restrict(self, self.Attribute_name)

        return self
    
    def Restrict(self, child, attribute_name):
        def _attributes_match(value):
            value_attributes = AttributesDict(value)
            if attribute_name in value_attributes:
                # v.p -> v.p ∩ child
                value_attribute = value_attributes[attribute_name]
                intersection = Subset(value_attribute, child)
                if intersection is None:
                    return False
                else:
                    # should this be a symbol?
                    setattr(value, attribute_name, intersection)
                    return True
            else:
                # child has not p
                return False

        new_vals = []
        for value in self.values:
            if _attributes_match(value):
                new_vals.append(value)
        if len(new_vals) == 0:
            raise Exception('empty restriction!')
        self.InitVals(new_vals)

        if self.parent is not None:
            self.parent.Restrict(self, self.Attribute_name)

def AttributesDict(obj):
    filters = [
        lambda key, value: key[0] != '_',
        lambda key, value: not callable(value),
        lambda key, value: key != 'parent' and key != 'values' and key != 'val'
    ]
    attributes = {i: obj.__getattribute__(i) for i in dir(obj) if i[0] != '_'}
    for fn in filters:
        attributes = {k: v for k, v in attributes.items() if fn(k, v)}
    return attributes

def Subset(value, target, depth=0):
    '''Return the subset of value that spans target. If no
    such subset exists, return None
    '''

    #######################
    ## VALUE IS ITERABLE ##
    #######################

    if type(value) is Symbol:
        subset = Subset(value.values, target, depth=depth+1)
        if subset is not None:
            return Symbol(subset)
        else:
            return None

    # TODO: explain why this works
    if type(value) is list:
        results = [Subset(element, target, depth=depth+1) for element in value]
        results = [result for result in results if result is not None]
        if len(results) == 0:
            return None
        if len(results) == 1:
            return results[0]
        return results
    
    ########################
    ## TARGET IS ITERABLE ##
    ########################

    if type(target) is Symbol:
        return Subset(value, target.values, depth=depth+1)

    # TODO: explain why this works
    if type(target) is list:
        for element in target:
            result = Subset(value, element, depth=depth+1)
            if result is not None:
                return result
        return None
    
    ###################
    # CONCRETE VALUES #
    ###################
    
    if value == target:
        return value
    if type(target) is type:
        if type(value) == target:
            return value
        if type(value) in target.__subclasses__():
            return value

    if type(value) not in PRIMITIVE_TYPES:
        if type(value) == type(target):
            return value
        if type(target) in type(value).__subclasses__():
            # TODO: (subclass)Value promotion
            # just take all attributes and methods
            return value

    # print('\t'*depth, is_subset)
    return None

In [2]:
import unittest

In [3]:
class Q(object):
    pass
class Q1(Q):
    pass
class Q2(Q):
    pass
class Q3(Q):
    pass

class P(object):
    pass
class P1(P):
    def __init__(self):
        self.Q = Q1()
class P2(P):
    def __init__(self):
        self.Q = Symbol([Q1(), Q2()])
class P3(P):
    def __init__(self):
        self.Q = Symbol([Q2(), Q3()])


class A(object):
    pass
class A1(A):
    def __init__(self):
        self.P = P1()
class A2(A):
    def __init__(self):
        self.P = Symbol([P1(), P2()])
class A3(A):
    def __init__(self):
        self.P = P3()

In [4]:
def SymbolIncludes(symbol, object_type):
    types = [type(value) for value in symbol.values]
    return object_type in types

def SymbolIncludesAll(symbol, types):
    return all([SymbolIncludes(symbol, t) for t in types])

def SymbolIncludesAny(symbol, types):
    return any([SymbolIncludes(symbol, t) for t in types])

In [61]:
class SymbolTest(unittest.TestCase):
    def setUp(self):
        self.A = Symbol(A)
        
    def collapse(self, symbol, val):
        self.setUp()
        symbol.Collapse(val)
        if val is type:
            self.assertTrue(SymbolIncludes(symbol, val))
        if val is list:
            self.assertTrue(SymbolIncludesAll(symbol, val))
    def collapse_Q(self, q):
#         self.collapse(self.A.P.Q, q)
        self.setUp()
        self.A.P.Q.Collapse(q)
        if q is type:
            self.assertTrue(SymbolIncludes(self.A.P.Q, q))
        if q is list:
            self.assertTrue(SymbolIncludesAll(self.A.P.Q, q))
    def collapse_P(self, p):
#         self.collapse(self.A.P, p)
        self.setUp()
        self.A.P.Collapse(p)
        if type(p) is type:
            self.assertTrue(SymbolIncludes(self.A.P, p))
        if type(p) is list:
            self.assertTrue(SymbolIncludesAll(self.A.P, p))
    def collapse_A(self, a):
#         self.collapse(self.A, a)
        self.setUp()
        self.A.Collapse(a)
        if type(a) is type:
            self.assertTrue(SymbolIncludes(self.A, a))
        if type(a) is list:
            self.assertTrue(SymbolIncludesAll(self.A, a))
    
class TestCollapse(SymbolTest):
    def test_A(self):        
        def _test_A1():
            self.collapse_A(A1)
            # P
            self.assertTrue(SymbolIncludes(self.A.P, P1))
            self.assertFalse(SymbolIncludesAny(self.A.P, [P2, P3]))
            # Q
            self.assertTrue(SymbolIncludes(self.A.P.Q, Q1))
            self.assertFalse(SymbolIncludesAny(self.A.P.Q, [Q2, Q3]))
        def _test_A2():
            self.collapse_A(A2)
            # P
            self.assertTrue(SymbolIncludesAll(self.A.P, [P1, P2]))
            self.assertFalse(SymbolIncludes(self.A.P, P3))
            # Q
            self.assertTrue(SymbolIncludesAll(self.A.P.Q, [Q1, Q2]))
            self.assertFalse(SymbolIncludes(self.A.P.Q, Q3))
        def _test_A3():
            self.collapse_A(A3)
            # P
            self.assertTrue(SymbolIncludes(self.A.P, P3))
            self.assertFalse(SymbolIncludesAny(self.A.P, [P1, P2]))
            # Q
            self.assertTrue(SymbolIncludesAll(self.A.P.Q, [Q2, Q3]))
            self.assertFalse(SymbolIncludes(self.A.P.Q, Q1))
        _test_A1()
        _test_A2()
        _test_A3()
    def test_P(self):        
        def _test_P1():
            self.collapse_P(P1)
            # A
            self.assertTrue(SymbolIncludesAll(self.A, [A1, A2]))
            self.assertFalse(SymbolIncludes(self.A, A3))
            # Q
            self.assertTrue(SymbolIncludes(self.A.P.Q, Q1))
            self.assertFalse(SymbolIncludesAny(self.A.P.Q, [Q2, Q3]))
        def _test_P2():
            self.collapse_P(P2)
            # A
            self.assertTrue(SymbolIncludes(self.A, A2))
            self.assertFalse(SymbolIncludesAny(self.A, [A1, A3]))
            # Q
            self.assertTrue(SymbolIncludesAll(self.A.P.Q, [Q1, Q2]))
            self.assertFalse(SymbolIncludes(self.A.P.Q, Q3))
        def _test_P3():
            self.collapse_P(P3)
            # A
            self.assertTrue(SymbolIncludes(self.A, A3))
            self.assertFalse(SymbolIncludesAny(self.A, [A1, A2]))
            # Q
            self.assertTrue(SymbolIncludesAll(self.A.P.Q, [Q2, Q3]))
            self.assertFalse(SymbolIncludes(self.A.P.Q, Q1))
        _test_P1()
        _test_P2()
        _test_P3()
    def test_Q(self):
        def _test_Q1():
            self.collapse_Q(Q1)
            # A
            self.assertTrue(SymbolIncludesAll(self.A, [A1, A2]))
            self.assertFalse(SymbolIncludes(self.A, A3))
            # P
            self.assertTrue(SymbolIncludesAll(self.A.P, [P1, P2]))
            self.assertFalse(SymbolIncludes(self.A.P, P3))
        def _test_Q2():
            self.collapse_Q(Q2)
            # A
            self.assertTrue(SymbolIncludesAll(self.A, [A2, A3]))
            self.assertFalse(SymbolIncludes(self.A, A1))
            # P
            self.assertTrue(SymbolIncludesAll(self.A.P, [P2, P3]))
            self.assertFalse(SymbolIncludes(self.A.P, P1))
        def _test_Q3():
            self.collapse_Q(Q3)
            # A
            self.assertTrue(SymbolIncludes(self.A, A3))
            self.assertFalse(SymbolIncludesAny(self.A, [A1, A2]))
            # P
            self.assertTrue(SymbolIncludes(self.A.P, P3))
            self.assertFalse(SymbolIncludesAny(self.A.P, [P1, P2]))
        _test_Q1()
        _test_Q2()
        _test_Q3()
    
unittest.main(argv=[''], verbosity=2, exit=False)

test_A (__main__.TestCollapse) ... ok
test_P (__main__.TestCollapse) ... ok
test_Q (__main__.TestCollapse) ... ok
test_A (__main__.TestMulti) ... FAIL
test_P (__main__.TestMulti) ... ok
test_Q (__main__.TestMulti) ... ok

FAIL: test_A (__main__.TestMulti)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-60-a198ff71d881>", line 40, in test_A
    self.collapse_A([A1, A2])
  File "<ipython-input-59-40fd2323567d>", line 33, in collapse_A
    self.assertTrue(SymbolIncludes(self.A, a))
AssertionError: False is not true

----------------------------------------------------------------------
Ran 6 tests in 0.019s

FAILED (failures=1)


In [86]:
class TestMulti(SymbolTest):
    def test_Q(self):
        self.collapse_Q([Q1, Q2])
        self.assertFalse(SymbolIncludes(self.A.P.Q, Q3))
        self.assertTrue(SymbolIncludesAll(self.A.P, [P1, P2, P3]))
        self.assertTrue(SymbolIncludesAll(self.A, [A1, A2, A3]))
        
        self.collapse_Q([Q1, Q3])
        self.assertFalse(SymbolIncludes(self.A.P.Q, Q2))
        self.assertTrue(SymbolIncludesAll(self.A.P, [P1, P2, P3]))
        self.assertTrue(SymbolIncludesAll(self.A, [A1, A2, A3]))
        
        self.collapse_Q([Q2, Q3])
        self.assertFalse(SymbolIncludes(self.A.P.Q, Q1))
        self.assertTrue(SymbolIncludesAll(self.A.P, [P2, P3]))
        self.assertFalse(SymbolIncludes(self.A.P, P1))
        self.assertTrue(SymbolIncludesAll(self.A, [A2, A3]))
        self.assertFalse(SymbolIncludes(self.A, A1))
        
    def test_P(self):
        self.collapse_P([P1, P2])
        self.assertFalse(SymbolIncludes(self.A.P, P3))
        self.assertTrue(SymbolIncludesAll(self.A.P.Q, [Q1, Q2]))
        self.assertFalse(SymbolIncludes(self.A.P.Q, Q3))
        self.assertTrue(SymbolIncludesAll(self.A, [A1, A2]))
        self.assertFalse(SymbolIncludes(self.A, A3))
        
        self.collapse_P([P1, P3])
        self.assertFalse(SymbolIncludes(self.A.P, P2))
        self.assertTrue(SymbolIncludesAll(self.A.P.Q, [Q1, Q2, Q3]))
        self.assertTrue(SymbolIncludesAll(self.A, [A1, A2, A3]))
        
        self.collapse_P([P2, P3])
        self.assertFalse(SymbolIncludes(self.A.P, P1))
        self.assertTrue(SymbolIncludesAll(self.A.P.Q, [Q1, Q2, Q3]))
        self.assertTrue(SymbolIncludesAll(self.A, [A2, A3]))
        self.assertFalse(SymbolIncludes(self.A, A1))
    
    def test_A(self):
        self.collapse_A([A1, A2])
        self.assertFalse(SymbolIncludes(self.A, A3))
        self.assertTrue(SymbolIncludesAll(self.A.P, [P1, P2]))
        self.assertFalse(SymbolIncludes(self.A.P, P3))
        self.assertTrue(SymbolIncludesAll(self.A.P.Q, [Q1, Q2]))
        self.assertFalse(SymbolIncludes(self.A.P.Q, Q3))
        
        self.collapse_A([A1, A3])
        self.assertFalse(SymbolIncludes(self.A, A2))
        self.assertTrue(SymbolIncludesAll(self.A.P.Q, [Q1, Q2, Q3]))
        self.assertTrue(SymbolIncludesAll(self.A.P, [P1, P3]))
        self.assertFalse(SymbolIncludes(self.A.P, P2))
        
        self.collapse_A([A2, A3])
        self.assertFalse(SymbolIncludes(self.A, A1))
        self.assertTrue(SymbolIncludesAll(self.A.P.Q, [Q1, Q2, Q3]))
        self.assertTrue(SymbolIncludesAll(self.A.P, [P1, P2, P3]))

unittest.main(argv=[''], verbosity=2, exit=False)

test_A (__main__.TestCollapse) ... ok
test_P (__main__.TestCollapse) ... ok
test_Q (__main__.TestCollapse) ... ok
test_A (__main__.TestMulti) ... ok
test_P (__main__.TestMulti) ... ok
test_Q (__main__.TestMulti) ... ok
runTest (__main__.TestSimultaneous) ... FAIL

FAIL: runTest (__main__.TestSimultaneous)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-85-ed0b32a41471>", line 15, in runTest
    self.assertFalse(SymbolIncludes(a.P, P1))
AssertionError: True is not false

----------------------------------------------------------------------
Ran 7 tests in 0.022s

FAILED (failures=1)


In [85]:
class TestSimultaneous(unittest.TestCase):
    def runTest(self):
        a = Symbol(A)
        a2 = Symbol(A)
        a2.Collapse([A2, A3])
        self.assertTrue(SymbolIncludesAll(a2, [A2, A3]))
        self.assertFalse(SymbolIncludes(a2, A1))
        a2.P.Q.Collapse(Q2)
        self.assertTrue(SymbolIncludesAll(a2.P, [P2, P3]))
        self.assertFalse(SymbolIncludes(a2.P, P1))
        self.assertTrue(SymbolIncludes(a2.P.Q, Q2))
        self.assertFalse(SymbolIncludesAny(a2.P.Q, [Q1, Q3]))
        a.Collapse(a2)
        # a2.P should restrict a.P
        self.assertFalse(SymbolIncludes(a.P, P1))
        # a2.P.Q should restrict a.P.Q
        self.assertFalse(SymbolIncludesAny(a.P.Q, [Q1, Q3]))
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_A (__main__.TestCollapse) ... ok
test_P (__main__.TestCollapse) ... ok
test_Q (__main__.TestCollapse) ... ok
test_A (__main__.TestMulti) ... ok
test_P (__main__.TestMulti) ... ok
test_Q (__main__.TestMulti) ... ok
runTest (__main__.TestSimultaneous) ... FAIL

FAIL: runTest (__main__.TestSimultaneous)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-85-ed0b32a41471>", line 15, in runTest
    self.assertFalse(SymbolIncludes(a.P, P1))
AssertionError: True is not false

----------------------------------------------------------------------
Ran 7 tests in 0.023s

FAILED (failures=1)


In [17]:
# test framework
#   how to split into separate files if autoreload is broken?
#   different object trees / minimal test tree
#   notebook vs standalone

# ✓ test collapse
# ✓ test selective collapse
# ✓ test collapsing to multiple values
# test collapsing simultaneous
# test improper collapse catching

# test attribute removal
# test attribute adoption